In [2]:
import yaml
import torch
import torch.nn.functional as F
from train_ptuning import KoGPTConditionalGeneration
from utils import generate_next_token

In [2]:
hparams_file = 'log/hparams.yaml'
with open(hparams_file) as f:
    hparams = yaml.load(f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [3]:
inf = KoGPTConditionalGeneration.load_from_checkpoint('./logs/model_chp/epoch=01-val_loss=2.282.ckpt')

/home/ubuntu/anaconda3/envs/torch1.1/lib/python3.8/site-packages/pytorch_lightning/utilities/argparse_utils.py:3: LightningDeprecationWarning: `argparse_utils` package has been renamed to `argparse` since v1.2 and will be removed in v1.4
  rank_zero_deprecation("`argparse_utils` package has been renamed to `argparse` since v1.2 and will be removed in v1.4")
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
tokenizer = inf.tokenizer

In [5]:
SUMMARY = '<unused1>'
PTUNING = '<unused2>'
EOS = '</s>'

In [21]:
text = """공주시 무령왕릉에서 출토된 백제 때 귀고리 2쌍으로 길이는 11.8㎝, 8.8㎝이다. 왕비의 귀고리로, 굵은 고리를 중심으로 작은 장식들을 연결하여 만들었다. 한 쌍은 복잡한 형식으로 길고 짧은 2줄의 장식이 달려 있고, 다른 한 쌍은 1줄로만 되어있다. 앞의 귀고리 중 긴 가닥은 금 철사를 꼬아서 만든 사슬에 둥근 장식을 많이 연결하였으며, 맨 밑에는 작은 고리를 연결하여 8개의 둥근 장식을 달고 그 아래 탄환 모양의 장식을 달았다. 짧은 줄의 수식은 다른 한 쌍의 것과 거의 같은 수법이나 탄환 장식은 달지 않고, 잎사귀 모양의 장식과 담록색의 둥근 옥을 달았다. 국립중앙박물관과 국립공주박물관에 각1쌍씩 보관되어 있다
"""

In [22]:
text = text.replace('\n', '')
input_tokens = tokenizer.encode(PTUNING)* 10 + tokenizer.encode(text) + tokenizer.encode(SUMMARY)
input_tensor = torch.tensor(input_tokens).unsqueeze(0)

eos_id = tokenizer.encode(EOS)[0]

In [23]:
cnt = 0
while True:
    if cnt > 128:
        break
    cnt += 1
    pred = inf.model(input_tensor)
    next_token = generate_next_token(pred.logits, temperature=1.0, top_p=0.8)

    if next_token.item() == eos_id:
        break
    else:
        input_tensor = torch.cat([input_tensor, next_token.unsqueeze(0)],1)


In [24]:
tokenizer.decode(input_tensor[0])

'<unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2><unused2> 공주시 무령왕릉에서 출토된 백제 때 귀고리 2쌍으로 길이는 11.8cm, 8.8cm이다. 왕비의 귀고리로, 굵은 고리를 중심으로 작은 장식들을 연결하여 만들었다. 한 쌍은 복잡한 형식으로 길고 짧은 2줄의 장식이 달려 있고, 다른 한 쌍은 1줄로만 되어있다. 앞의 귀고리 중 긴 가닥은 금 철사를 꼬아서 만든 사슬에 둥근 장식을 많이 연결하였으며, 맨 밑에는 작은 고리를 연결하여 8개의 둥근 장식을 달고 그 아래 탄환 모양의 장식을 달았다. 짧은 줄의 수식은 다른 한 쌍의 것과 거의 같은 수법이나 탄환 장식은 달지 않고, 잎사귀 모양의 장식과 담록색의 둥근 옥을 달았다. 국립중앙박물관과 국립공주박물관에 각1쌍씩 보관되어 있다<unused1> 왕비의 귀고리로, 굵은 고리를 중심으로 작은 장식들을 연결하여 만들었다.  왕비의 귀고리로, 굵은 고리를 중심으로 작은 장식들을 연결하였으며, 굵은 고리를 중심으로 작은 장식들을 연결하여 만들었다.\n육십팔사 십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.8cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.9cm\n육십팔사 평하: 11.8cm\n'

In [20]:
tokenizer.decode(input_tensor[0]).split('<unused1>')[-1].strip()

'왕비가 귀고리로는 지름 5cm의 장식이 달린 두 줄을 달려 있고 다른 한 쌍의 것과 거의 비슷한 수법과 탄환 장식은 달지 않았다.  작은 매듭은 단단한 형태로 길고 긴 매듭을 많이 연결했으며, 접은 유리 모양을 한 작은 천으로 장식한 것으로 둥글게 부풀어 오르며 세로로 길게 늘어나 있다.  왕비의 귀고리로는 지름 5cm의 장식이 달린 두 줄을 달리고 가로로 길게 늘어졌다.  다발에 의한 큰 장식들은 금 철사를 꼬아서 만든 사슬에 둥근 장식을 많이 연결하였고, 그 위에 탄환 모양의 장식을 많이 달았다.  다음은 이와 같다.  왕실 박물관과 국립공주박물관에 각1쌍씩 보관되어 있다涑 왕이 만든 가죽으로 만든 접이다.  궁내 공주의 귀고리로는 지름 5cm의 장식이 달린 두 줄을 달았으며, 접은 유리 모양을 한 작은 천으로 장식하였다.  왕비의 귀고리로는 지름 5cm의 장식이 달린 두 줄을 달았으며, 접은 유리 모양을 한 큰 천으로 장식하였다.  왕비가 귀고리로는 지름 5cm의 장식과 담록색의 둥근 옥을 달았으며, 접은 유리 모양을 한 작은 천으로 장식하였다.  고종이 만든 가죽으로 만든 접은 유리 모양을 한 작은 천으로 장식하였다.  궁궐과 공주에 각1쌍씩 보관되어 있다 3칸의 장식과 담록색의 둥근 옥을 달았다.  고종의 귀고리로는 지름 5cm의 장식이 달린 두 줄을 달았으며, 접은 유리 모양을 한 큰 천으로 장식하였다.  문무 공주와 궁궐의 주빈에 각1쌍씩 보관되어 있다 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중수 중